In [26]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder

In [27]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful AI Chatbot and you talk with user in {language} langugae. User's name is {name}"),
    HumanMessagePromptTemplate.from_template("User will talk friendly to Chatbot"),
    MessagesPlaceholder(variable_name="messages")
])

In [28]:
load_dotenv()

True

In [29]:
llm = ChatGroq(temperature=0, model='llama3-8b-8192')

In [30]:
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages, Sequence

In [31]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
    name: str

In [32]:
def model(state: State):
    msg = prompt_template.invoke({"messages": state['messages'], "language": state['language'], "name": state["name"]})
    response = llm.invoke(msg)
    return {"messages": [response]}

In [33]:
from langgraph.graph import StateGraph
from langgraph.graph import START, END
from langgraph.checkpoint.memory import MemorySaver

In [34]:
graph = StateGraph(state_schema=State)

graph.add_edge(START, "model")
graph.add_node("model", model)

memory = MemorySaver()

app = graph.compile(checkpointer=memory)

In [35]:
config = {"configurable": {"thread_id": "user123"}}
query = "My name is Prabal"
language  = "hindi"
name = "Pro"
input_msg = [HumanMessage(query)]

In [36]:
app.invoke({"messages": input_msg, "language": language, "name": name}, config)

{'messages': [HumanMessage(content='My name is Prabal', additional_kwargs={}, response_metadata={}, id='43cc98c9-3999-4026-b303-4f366f01706d'),
  AIMessage(content='Prabal ji! मैं आपका स्वागत करता हूँ! (I welcome you!) क्या आप मुझसे कुछ बातचीत करना चाहते हैं? (What do you want to talk about?)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 60, 'total_tokens': 113, 'completion_time': 0.068339368, 'prompt_time': 0.019369034, 'queue_time': 0.267283516, 'total_time': 0.087708402}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8af39bf2ae', 'finish_reason': 'stop', 'logprobs': None}, id='run--3c6f6fae-81eb-45da-842b-f228807cb1ff-0', usage_metadata={'input_tokens': 60, 'output_tokens': 53, 'total_tokens': 113})],
 'language': 'hindi',
 'name': 'Pro'}

In [37]:
from langgraph.prebuilt import create_react_agent

In [38]:
!pip install langchain-tavily

In [39]:
from langchain_tavily import TavilySearch

In [51]:
def get_weather(city: str):
    """Get weather for given city"""
    return f"It is sunny in {city}"

In [52]:
search = TavilySearch(max_results=2)

In [57]:
tools = [get_weather]

In [59]:
agent_executor = create_react_agent(model=llm, tools=tools)

In [60]:
query1 = {"role": "user", "content": "What is the weather in Indore India?"}
config1 = {"configurable": {"thread_id": "user123"}}

In [61]:
agent_executor.invoke({"messages": query1}, config1)["messages"][-1].content

'It is sunny in Indore.'